In [ ]:
# Version 2 - Mi mejor modelo (MCC 0.8885 publico)
# Aprendí de V1 y añadí regularización L2 fuerte
# Este fue el modelo que nunca superé en el leaderboard publico
# Aunque V6 es mejor en privado por estabilidad

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import sys
try:
    import google.protobuf
    if hasattr(google.protobuf, '__version__'):
        print(f"Protobuf version: {google.protobuf.__version__}")
except:
    pass

print("✓ Environment variables set successfully")
print("="*60)

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seeds BEFORE importing TensorFlow
seed = 42
np.random.seed(seed)

print("="*60)
print("IMPORTING TENSORFLOW AND KERAS...")
print("="*60)

# Import TensorFlow
import tensorflow as tf

# Force TensorFlow to use compatible protobuf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import from tensorflow.keras (NOT standalone keras)
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, 
    GlobalMaxPooling1D, Conv1D, SpatialDropout1D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2  # ITER2: NUEVO - Regularización L2

# Set TensorFlow seeds
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

# Sklearn metrics
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Display settings
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

print("="*60)
print("ENVIRONMENT SETUP - SUCCESS!")
print("="*60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if tf.config.list_physical_devices('GPU'):
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(f"GPU Devices: {len(gpu_devices)} device(s)")
    for gpu in gpu_devices:
        print(f"  - {gpu.name}")
print(f"Random seed: {seed}")
print("="*60)

## Iteracion 2 - Aprendí de V1 y ataqué el overfitting

**Mi problema en V1:**
Train Loss era super bajo (0.0055) pero Val Loss alto (0.1895) → Overfitting brutal

**Lo que cambié:**
- Reducí LSTM de 128 a 96 unidades (-25% capacidad)
- Reduje Dense de 64 a 48 (-25% capacidad)
- Subi Dropout de 0.5 a 0.6
- Subi Spatial Dropout de 0.2 a 0.3
- Añadí regularización L2 (1e-4) - NUEVO

**Objetivo:** Delta entre train y val loss menor que 0.10

In [ ]:
# Hyperparameters Configuration - ITERACIÓN 2
MAX_WORDS = 10000  # Vocabulario máximo
MAX_LEN = 200  # Longitud máxima de secuencias
EMBEDDING_DIM = 100  # Dimensión de embeddings (compatible con GloVe-100d)

# Model Architecture - CAMBIOS PARA REDUCIR OVERFITTING
LSTM_UNITS = 96           # ITER2: CAMBIO 128 → 96 (-25%)
DENSE_UNITS = 48          # ITER2: CAMBIO 64 → 48 (-25%)
DROPOUT_RATE = 0.6        # ITER2: CAMBIO 0.5 → 0.6 (+20%)
SPATIAL_DROPOUT = 0.3     # ITER2: CAMBIO 0.2 → 0.3 (+50%)
L2_REG = 1e-4             # ITER2: NUEVO - Regularización L2

# Training Configuration
BATCH_SIZE = 32
EPOCHS = 50
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 1e-3

print("="*60)
print("MODEL CONFIGURATION - ITERACIÓN 2")
print("="*60)
print("🎯 OBJETIVO: Reducir overfitting manteniendo MCC > 0.86")
print("="*60)
print("CAMBIOS respecto a Iteración 1:")
print("  - LSTM Units: 128 → 96 (-25%)")
print("  - Dense Units: 64 → 48 (-25%)")
print("  - Spatial Dropout: 0.2 → 0.3 (+50%)")
print("  - Dropout: 0.5 → 0.6 (+20%)")
print("  - L2 Regularization: None → 1e-4 (NUEVO)")
print("  - Early Stopping Patience: 5 → 3")
print("="*60)
print(f"Vocabulary Size: {MAX_WORDS:,}")
print(f"Sequence Length: {MAX_LEN}")
print(f"Embedding Dimension: {EMBEDDING_DIM}")
print(f"LSTM Units: {LSTM_UNITS}")
print(f"Dense Units: {DENSE_UNITS}")
print(f"Dropout Rate: {DROPOUT_RATE}")
print(f"Spatial Dropout: {SPATIAL_DROPOUT}")
print(f"L2 Regularization: {L2_REG}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Epochs: {EPOCHS}")
print("="*60)

# Estimación de parámetros esperados
estimated_params = (
    MAX_WORDS * EMBEDDING_DIM +  # Embedding
    4 * LSTM_UNITS * (EMBEDDING_DIM + LSTM_UNITS + 1) * 2 +  # Bi-LSTM
    (LSTM_UNITS * 2) * DENSE_UNITS + DENSE_UNITS +  # Dense
    DENSE_UNITS + 1  # Output
)
print(f"\n📊 Parámetros estimados: ~{estimated_params:,}")
print(f"   (V1 tenía: 1,251,009 parámetros)")
print(f"   Reducción esperada: ~{((1251009 - estimated_params) / 1251009 * 100):.1f}%")
print("="*60)

## Carga y Exploración de Datos

Misma exploración que Iteración 1 para mantener consistencia.

In [ ]:
# Load training data
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

print("="*60)
print("TRAINING DATA OVERVIEW")
print("="*60)
print(f"Total samples: {len(train):,}")
print(f"\nColumns: {list(train.columns)}")
print(f"\nData types:\n{train.dtypes}")
print(f"\nNull values:\n{train.isnull().sum()}")
print(f"\nClass distribution:\n{train['spam_label'].value_counts()}")
print(f"\nClass balance:\n{train['spam_label'].value_counts(normalize=True)}")
print("="*60)

# Display sample data
print("\nSample data:")
train.head(10)

In [ ]:
# Análisis de longitud de textos
train['text_length'] = train['text'].apply(lambda x: len(str(x).split()))

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(train['text_length'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Longitud de Textos')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.axvline(train['text_length'].mean(), color='red', linestyle='--', label=f'Media: {train["text_length"].mean():.1f}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
train.groupby('spam_label')['text_length'].hist(bins=30, alpha=0.7, label=['Not SPAM', 'SPAM'])
plt.title('Longitud por Clase')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
sns.boxplot(data=train, x='spam_label', y='text_length')
plt.title('Boxplot Longitud por Clase')
plt.xlabel('Clase (0=Not SPAM, 1=SPAM)')
plt.ylabel('Número de palabras')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Longitud promedio: {train['text_length'].mean():.2f} palabras")
print(f"Longitud mediana: {train['text_length'].median():.2f} palabras")
print(f"Longitud máxima: {train['text_length'].max()} palabras")
print(f"Longitud mínima: {train['text_length'].min()} palabras")

## Preprocesamiento de Texto

Tokenización y preparación de secuencias para el modelo.

In [ ]:
# Preparar textos y etiquetas
X_train_text = train['text'].values
y_train = train['spam_label'].values

# Tokenización
print("Tokenizando textos...")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Convertir textos a secuencias
X_train_seq = tokenizer.texts_to_sequences(X_train_text)

# Padding de secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Split train/validation
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed,
    stratify=y_train
)

print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)
print(f"Vocabulary size: {len(tokenizer.word_index):,}")
print(f"Training samples: {len(X_train_final):,}")
print(f"Validation samples: {len(X_val):,}")
print(f"Sequence shape: {X_train_pad.shape}")
print(f"Train class distribution: {np.bincount(y_train_final)}")
print(f"Val class distribution: {np.bincount(y_val)}")
print("="*60)

## 🔧 Construcción del Modelo - ITERACIÓN 2

**Arquitectura Modificada:**
- Embedding → Spatial Dropout (↑0.3) → Bidirectional LSTM (↓96) + L2 → GlobalMaxPooling → Dense (↓48) + L2 → Dropout (↑0.6) → Output

**Mejoras de Regularización:**
1. ✅ Menos unidades LSTM (96 vs 128) - Reduce capacidad
2. ✅ Menos unidades Dense (48 vs 64) - Reduce capacidad  
3. ✅ Spatial Dropout aumentado (0.3 vs 0.2) - Más regularización en embeddings
4. ✅ Dropout aumentado (0.6 vs 0.5) - Más regularización en capa densa
5. ✅ L2 Regularization (1e-4) - **NUEVO** - Penaliza pesos grandes

**Objetivo:** Forzar al modelo a generalizar en lugar de memorizar

In [ ]:
# Construcción del modelo V2 con regularización mejorada
def build_model_v2():
    model = Sequential([
        # Embedding layer
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            name='embedding'
        ),
        
        # ITER2: Aumentar Spatial Dropout 0.2 → 0.3
        SpatialDropout1D(SPATIAL_DROPOUT),
        
        # ITER2: Reducir LSTM units 128 → 96 + Añadir L2 regularization
        Bidirectional(
            LSTM(
                LSTM_UNITS, 
                return_sequences=True,
                kernel_regularizer=l2(L2_REG),      # NUEVO
                recurrent_regularizer=l2(L2_REG)    # NUEVO
            ), 
            name='bidirectional_lstm'
        ),
        
        # Global Max Pooling para extraer características más relevantes
        GlobalMaxPooling1D(),
        
        # ITER2: Reducir Dense units 64 → 48 + Añadir L2 regularization
        Dense(
            DENSE_UNITS, 
            activation='relu',
            kernel_regularizer=l2(L2_REG),          # NUEVO
            name='dense_1'
        ),
        
        # ITER2: Aumentar Dropout 0.5 → 0.6
        Dropout(DROPOUT_RATE),
        
        # Output layer
        Dense(1, activation='sigmoid', name='output')
    ], name='spam_classifier_v2')
    
    return model

# Crear modelo
model = build_model_v2()

# Compilar con AdamW y binary crossentropy
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

# Build the model explicitly before showing summary
model.build(input_shape=(None, MAX_LEN))

# Resumen del modelo
model.summary()

print("\n" + "="*60)
print("MODEL V2 COMPILED - OVERFITTING REDUCTION MODE")
print("="*60)
print(f"Total parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")
print(f"\n📉 Comparación con V1:")
print(f"   V1 parámetros: 1,251,009")
print(f"   V2 parámetros: {model.count_params():,}")
print(f"   Reducción: {((1251009 - model.count_params()) / 1251009 * 100):.1f}%")
print(f"\nOptimizer: AdamW (lr={LEARNING_RATE}, weight_decay=1e-4)")
print(f"Loss: Binary Crossentropy")
print(f"\n🎯 Objetivo: (Val Loss - Train Loss) < 0.10")
print("="*60)

## Entrenamiento del Modelo - ITERACIÓN 2

**Callbacks Modificados:**
- **EarlyStopping**: Patience 5 → **3** (más agresivo)
- **ReduceLROnPlateau**: Patience 3 → **2** (reduce LR más rápido)
- **ModelCheckpoint**: Guarda mejor modelo

**Expectativa:** El modelo debería detenerse antes, evitando memorización excesiva.

In [ ]:
# ITER2: Callbacks con early stopping más agresivo
callbacks = [
    # ITER2: Patience 5 → 3 para detener más rápido
    EarlyStopping(
        monitor='val_loss',
        patience=3,  # CAMBIO: 5 → 3
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model_v2.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    # ITER2: Patience 3 → 2 para reducir LR más rápido
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,  # CAMBIO: 3 → 2
        min_lr=1e-6,
        verbose=1
    )
]

# Entrenamiento
print("="*60)
print("STARTING TRAINING - ITERACIÓN 2")
print("="*60)
print("🎯 Monitoreando overfitting...")
print("   Esperando: Train Loss > 0.02 y Val Loss < 0.15")
print("   Meta: (Val Loss - Train Loss) < 0.10")
print("="*60)

history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETED - ITERACIÓN 2")
print("="*60)

## Evaluación del Modelo

Análisis de métricas en el conjunto de validación.

In [ ]:
# Predicciones en validación
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Calcular Matthews Correlation Coefficient (métrica de la competición)
mcc_score = matthews_corrcoef(y_val, y_pred)

print("="*60)
print("VALIDATION METRICS - ITERACIÓN 2")
print("="*60)
print(f"Matthews Correlation Coefficient: {mcc_score:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*60)

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'Confusion Matrix V2 (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 📊 Comparación V1 vs V2 - Análisis de Overfitting

**Objetivo:** Verificar si redujimos el overfitting exitosamente.

In [ ]:
# Análisis de overfitting V2
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
overfitting_delta = abs(final_val_loss - final_train_loss)

print("\n" + "="*60)
print("ANÁLISIS DE OVERFITTING - V1 vs V2")
print("="*60)

# Comparación con V1
v1_train_loss = 0.0055
v1_val_loss = 0.1895
v1_delta = abs(v1_val_loss - v1_train_loss)

print("\n📊 ITERACIÓN 1 (Baseline):")
print(f"   Train Loss: {v1_train_loss:.4f}")
print(f"   Val Loss: {v1_val_loss:.4f}")
print(f"   Overfitting Δ: {v1_delta:.4f} ⚠️")

print("\n📊 ITERACIÓN 2 (Reducción Overfitting):")
print(f"   Train Loss: {final_train_loss:.4f}")
print(f"   Val Loss: {final_val_loss:.4f}")
print(f"   Overfitting Δ: {overfitting_delta:.4f}")

print("\n📈 MEJORA:")
delta_improvement = ((v1_delta - overfitting_delta) / v1_delta * 100)
print(f"   Reducción de overfitting: {delta_improvement:.1f}%")

if overfitting_delta < 0.10:
    print("   ✅ ÉXITO: Overfitting bajo control (Δ < 0.10)")
elif overfitting_delta < v1_delta:
    print(f"   ⚠️ MEJORA PARCIAL: Overfitting reducido pero aún alto")
else:
    print("   ❌ SIN MEJORA: Se necesitan cambios más agresivos")

print("\n📊 Accuracy:")
print(f"   V1: Train {0.9991:.4f} | Val {0.9577:.4f} | Δ {0.0414:.4f}")
print(f"   V2: Train {final_train_acc:.4f} | Val {final_val_acc:.4f} | Δ {abs(final_val_acc - final_train_acc):.4f}")

print("\n📊 MCC Comparison:")
v1_mcc = 0.8665
print(f"   V1 MCC: {v1_mcc:.4f}")
print(f"   V2 MCC: {mcc_score:.4f}")
mcc_change = mcc_score - v1_mcc
if mcc_change >= 0:
    print(f"   ✅ Cambio: +{mcc_change:.4f} (Mejoró)")
else:
    print(f"   ⚠️ Cambio: {mcc_change:.4f} (Empeoró ligeramente - aceptable si overfitting mejoró)")

print("="*60)

## Curvas de Aprendizaje

Análisis visual de la evolución del entrenamiento.

In [ ]:
# Visualización de curvas de aprendizaje
def plot_learning_curves(history, title_prefix=""):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title(f'{title_prefix} Loss', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 1].set_title(f'{title_prefix} Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].set_title(f'{title_prefix} Precision', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 1].set_title(f'{title_prefix} Recall', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_learning_curves(history, title_prefix="Model V2")

## Predicciones en Test Data

Generación de predicciones para el conjunto de test de la competición.

In [ ]:
# Load test data
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

print("="*60)
print("TEST DATA")
print("="*60)
print(f"Total test samples: {len(test):,}")
print("="*60)

test.head()

In [ ]:
# Preprocesar test data
X_test_text = test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test_text)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Generar predicciones
print("Generando predicciones en test data...")
y_pred_proba_test = model.predict(X_test_pad, batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()

print(f"Predicciones generadas: {len(y_pred_test):,}")
print(f"Distribución de predicciones:")
print(f"  Not SPAM (0): {np.sum(y_pred_test == 0):,} ({np.mean(y_pred_test == 0)*100:.2f}%)")
print(f"  SPAM (1): {np.sum(y_pred_test == 1):,} ({np.mean(y_pred_test == 1)*100:.2f}%)")

## Generación del Archivo de Submission

Creación del archivo `submission_v2.csv` para envío a Kaggle.

In [ ]:
# Crear submission file
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)  # Kaggle espera 'submission.csv'

print("="*60)
print("SUBMISSION FILE CREATED - ITERACIÓN 2")
print("="*60)
print(f"Total predictions: {len(submission):,}")
print(f"File: submission.csv")
print("="*60)

# Verificar submission
print("\nPrimeras predicciones:")
submission.head(10)

---

# 🎯 Resumen Final - Iteración 2

## Objetivos y Resultados

In [ ]:
# Resumen final comparativo
comparison_df = pd.DataFrame({
    'Métrica': [
        'Validation MCC',
        'Train Loss',
        'Val Loss',
        'Overfitting Δ',
        'Train Accuracy',
        'Val Accuracy',
        'Total Parameters',
        'Training Time (epochs)'
    ],
    'Iteración 1': [
        '0.8665',
        '0.0055',
        '0.1895',
        '0.1840 ⚠️',
        '99.91%',
        '95.77%',
        '1,251,009',
        '8 epochs'
    ],
    'Iteración 2': [
        f'{mcc_score:.4f}',
        f'{final_train_loss:.4f}',
        f'{final_val_loss:.4f}',
        f'{overfitting_delta:.4f}',
        f'{final_train_acc:.2%}',
        f'{final_val_acc:.2%}',
        f'{model.count_params():,}',
        f'{len(history.history["loss"])} epochs'
    ]
})

print("="*80)
print("RESUMEN COMPARATIVO - ITERACIÓN 1 vs ITERACIÓN 2")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Análisis de éxito
print("\n🎯 ANÁLISIS DE ÉXITO:")
print("="*80)

success_criteria = [
    ("Overfitting Δ < 0.10", overfitting_delta < 0.10),
    ("Val MCC >= 0.86", mcc_score >= 0.86),
    ("Reducción de parámetros", model.count_params() < 1251009),
    ("Val Loss mejoró", final_val_loss < 0.1895)
]

for criterion, success in success_criteria:
    status = "✅" if success else "❌"
    print(f"{status} {criterion}")

total_success = sum([s for _, s in success_criteria])
print("\n" + "="*80)
print(f"CRITERIOS CUMPLIDOS: {total_success}/4")

if total_success >= 3:
    print("\n🎉 ITERACIÓN 2 EXITOSA - Overfitting reducido exitosamente")
    print("\n📝 PRÓXIMO PASO: Iteración 3 con GloVe embeddings pre-entrenados")
elif total_success >= 2:
    print("\n⚠️ MEJORA PARCIAL - Considerar ajustes adicionales")
    print("\n📝 PRÓXIMO PASO: Aumentar más dropout o probar técnicas avanzadas")
else:
    print("\n❌ NECESITA MÁS TRABAJO - Cambios más agresivos requeridos")
    print("\n📝 PRÓXIMO PASO: Revisar arquitectura completa")

print("="*80)

## 📚 Conclusiones de Iteración 2

### Cambios Implementados:
1. ✅ **Capacidad reducida**: -25% unidades (LSTM 128→96, Dense 64→48)
2. ✅ **Regularización aumentada**: Dropout +50% spatial, +20% dense
3. ✅ **L2 Regularization**: 1e-4 en LSTM y Dense (NUEVO)
4. ✅ **Early stopping agresivo**: Patience 5→3
5. ✅ **Parámetros reducidos**: ~15-20% menos parámetros totales

### Hipótesis Verificada:
- **Hipótesis**: Reducir capacidad + aumentar regularización → menos overfitting
- **Resultado**: [Se completará con ejecución real]

### Próximas Iteraciones:
- **Iteración 3**: GloVe embeddings pre-entrenados
- **Iteración 4**: CNN + LSTM híbrido
- **Iteración 5**: Ensemble de mejores modelos

### Referencias:
- L2 Regularization: https://keras.io/api/layers/regularizers/
- Dropout Paper: Srivastava et al. (2014) "Dropout: A Simple Way to Prevent Neural Networks from Overfitting"
- Overfitting Solutions: https://machinelearningmastery.com/introduction-to-regularization-to-reduce-overfitting-and-improve-generalization-error/